# 🤖 Agent & Tools - 도구를 사용하는 AI 에이전트

이 노트북은 beanllm의 Agent와 Tools 기능을 학습합니다. Agent는 Tool을 사용하여 복잡한 작업을 수행할 수 있습니다.

## 📋 목차

- [1. Tool 기본 사용](#1-tool-기본-사용)
- [2. 커스텀 Tool 생성](#2-커스텀-tool-생성)
- [3. Agent 기본 사용](#3-agent-기본-사용)
- [4. ReAct 패턴](#4-react-패턴)
- [5. 고급 Tool 기능](#5-고급-tool-기능)
- [6. 실전 예제](#6-실전-예제)

---

## 🎯 학습 목표

이 노트북을 완료하면:
- ✅ Tool을 정의하고 사용할 수 있습니다
- ✅ Agent를 생성하고 실행할 수 있습니다
- ✅ ReAct 패턴을 이해할 수 있습니다
- ✅ 여러 Tool을 조합하여 복잡한 작업을 수행할 수 있습니다
- ✅ Tool 검증과 에러 처리를 할 수 있습니다

---

## 📚 사전 요구사항

- [01_setup_and_installation.ipynb](01_setup_and_installation.ipynb) 완료
- [02_core_client.ipynb](02_core_client.ipynb) 완료
- 최소 하나의 LLM Provider API 키 설정


## 1. Tool 기본 사용

Tool은 Agent가 사용할 수 있는 함수입니다. 계산기, 웹 검색, API 호출, 데이터베이스 쿼리 등 다양한 기능을 제공할 수 있습니다. Tool을 사용하면 Agent는 단순한 텍스트 생성뿐만 아니라 실제 작업을 수행할 수 있습니다.

> 💡 **Tool의 핵심**:
> - **함수 변환**: 일반 Python 함수를 Tool로 변환할 수 있습니다
> - **스키마 정의**: Tool은 이름, 설명, 파라미터 스키마를 가집니다
> - **자동 선택**: Agent가 LLM을 사용하여 적절한 Tool을 자동으로 선택합니다
> - **타입 안전성**: Pydantic을 사용하여 파라미터 타입을 검증합니다

> 💡 **Tool의 구조**:
> - `name`: Tool의 이름 (함수 이름에서 자동 생성)
> - `description`: Tool의 설명 (LLM이 Tool을 선택하는 데 사용)
> - `parameters`: 파라미터 스키마 (JSON Schema 형식)
> - `function`: 실제 실행할 함수


In [ ]:
from beanllm import Tool, tool
import json

# ============================================
# Tool 기본 사용
# ============================================

print("=" * 60)
print("Tool 기본 사용")
print("=" * 60)

# 방법 1: 함수에서 Tool 생성
print("\n[방법 1] 함수에서 Tool 생성:")


def calculate(expression: str) -> str:
    """
    수식을 계산합니다.

    이 함수는 간단한 수학 표현식을 계산합니다.
    실제 프로덕션에서는 더 안전한 방법을 사용해야 합니다.

    Args:
        expression: 계산할 수식 (예: "2 + 2", "10 * 5", "100 / 4")

    Returns:
        계산 결과를 문자열로 반환
    """
    try:
        # 실제로는 ast.literal_eval() 또는 수식 파서 사용 권장
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"오류: {str(e)}"


# Tool 생성
calc_tool = Tool.from_function(calculate)

print("  ✅ Tool 생성 완료!")
print(f"  📊 이름: {calc_tool.name}")
print(f"  📊 설명: {calc_tool.description}")
print(f"  📊 파라미터 스키마:")
print(f"    {json.dumps(calc_tool.parameters, indent=4, ensure_ascii=False)}")

# Tool 직접 실행
print("\n[Tool 직접 실행]")
result = calc_tool.run(expression="2 + 2 * 3")
print(f"  입력: 2 + 2 * 3")
print(f"  결과: {result}")

# 여러 계산 테스트
test_expressions = ["10 + 20", "100 / 4", "2 ** 8", "sqrt(16)"]  # 이건 실패할 것

print("\n[여러 계산 테스트]")
for expr in test_expressions:
    try:
        result = calc_tool.run(expression=expr)
        print(f"  {expr} = {result}")
    except Exception as e:
        print(f"  {expr} = 오류: {e}")

print("\n💡 Tool의 특징:")
print("  - 함수를 자동으로 Tool로 변환")
print("  - Docstring에서 설명 자동 추출")
print("  - 타입 힌트에서 파라미터 스키마 자동 생성")
print("  - Agent가 이해할 수 있는 형식으로 변환")

# ============================================
# Tool 스키마 상세 분석
# ============================================

print("\n" + "=" * 60)
print("Tool 스키마 상세 분석")
print("=" * 60)

print("\n📊 Tool 객체 구조:")
print(f"  - name: {calc_tool.name}")
print(f"  - description: {calc_tool.description[:50]}...")
print(f"  - parameters type: {type(calc_tool.parameters)}")

print("\n📊 파라미터 스키마 상세:")
if isinstance(calc_tool.parameters, dict):
    schema = calc_tool.parameters
    print(f"  - type: {schema.get('type', 'N/A')}")
    print(f"  - properties: {list(schema.get('properties', {}).keys())}")
    print(f"  - required: {schema.get('required', [])}")

print("\n💡 Tool 스키마의 역할:")
print("  - LLM이 Tool을 이해하는 데 사용")
print("  - 파라미터 타입 검증")
print("  - 자동 완성 및 문서화")
print("  - Agent가 올바른 파라미터로 호출")

print("\n" + "=" * 60)
print("✅ Tool 기본 사용 학습 완료!")
print("=" * 60)

### 1.1 @tool 데코레이터

함수에 `@tool` 데코레이터를 사용하면 자동으로 Tool로 변환됩니다. 이는 가장 간편한 방법이며, 함수 정의와 동시에 Tool을 생성할 수 있습니다.

> 💡 **@tool 데코레이터의 장점**:
> - 간편한 사용: 함수 정의만으로 Tool 생성
> - 자동 변환: 함수 메타데이터에서 자동으로 정보 추출
> - 깔끔한 코드: 별도의 Tool 생성 코드 불필요
> - 타입 안전성: 타입 힌트를 자동으로 스키마로 변환


In [ ]:
from beanllm import tool
from typing import List

# ============================================
# @tool 데코레이터 사용
# ============================================

print("=" * 60)
print("@tool 데코레이터 사용")
print("=" * 60)


# @tool 데코레이터 사용 예제
@tool
def get_weather(city: str, unit: str = "celsius") -> str:
    """
    특정 도시의 날씨 정보를 가져옵니다.

    Args:
        city: 도시 이름 (예: "서울", "부산", "New York")
        unit: 온도 단위 ("celsius" 또는 "fahrenheit")

    Returns:
        날씨 정보 문자열
    """
    # 실제로는 날씨 API를 호출하지만, 여기서는 시뮬레이션
    return f"{city}의 날씨는 20도({unit})입니다. 맑음."


print("\n✅ @tool 데코레이터로 Tool 생성 완료!")
print(f"📊 함수 이름: {get_weather.__name__}")

# Tool로 사용
# weather_tool = get_weather  # 이미 Tool 객체
# result = weather_tool.run(city="서울", unit="celsius")
# print(f"\n🌤️ 날씨 조회 결과: {result}")

print("\n💡 @tool 데코레이터의 특징:")
print("  - 함수 정의와 동시에 Tool 생성")
print("  - Docstring에서 설명 자동 추출")
print("  - 타입 힌트에서 파라미터 스키마 생성")
print("  - 기본값이 있는 파라미터는 선택적 파라미터로 처리")

# ============================================
# 복잡한 Tool 예제
# ============================================

print("\n" + "=" * 60)
print("복잡한 Tool 예제")
print("=" * 60)


@tool
def search_database(query: str, table: str, limit: int = 10, filters: dict = None) -> List[dict]:
    """
    데이터베이스에서 데이터를 검색합니다.

    Args:
        query: 검색 쿼리
        table: 테이블 이름
        limit: 반환할 최대 결과 수 (기본값: 10)
        filters: 추가 필터 딕셔너리 (선택적)

    Returns:
        검색 결과 리스트
    """
    # 실제로는 데이터베이스 쿼리를 실행하지만, 여기서는 시뮬레이션
    results = [
        {"id": 1, "name": "결과 1", "value": 100},
        {"id": 2, "name": "결과 2", "value": 200},
    ]
    return results[:limit]


print("\n✅ 복잡한 Tool 생성 완료!")
print(f"📊 함수: {search_database.__name__}")

# Tool 사용 예제
# db_tool = search_database
# results = db_tool.run(
#     query="사용자 정보",
#     table="users",
#     limit=5
# )
# print(f"\n📊 검색 결과: {len(results)}개")

print("\n💡 복잡한 Tool 작성 팁:")
print("  - 명확한 Docstring 작성 (LLM이 이해하는 데 중요)")
print("  - 타입 힌트 명시 (자동 스키마 생성)")
print("  - 기본값 사용 (선택적 파라미터)")
print("  - 에러 처리 포함")

print("\n" + "=" * 60)
print("✅ @tool 데코레이터 학습 완료!")
print("=" * 60)


def get_weather(city: str) -> str:
    """도시의 날씨를 조회합니다.

    Args:
        city: 도시 이름 (예: "서울", "부산")

    Returns:
        날씨 정보
    """
    # 실제로는 API 호출
    return f"{city}의 날씨는 맑습니다. 온도: 20°C"


# 자동으로 Tool로 변환됨
print("✅ @tool 데코레이터로 Tool 생성")
print(f"📊 이름: {get_weather.name}")
print(f"📊 설명: {get_weather.description}")

# 실행
result = get_weather.run(city="서울")
print(f"\n🌤️ 날씨: {result}")

### 1.2 내장 Tool

beanllm은 여러 내장 Tool을 제공합니다.


In [ ]:
from beanllm import calculator, get_current_time, search_web

# 내장 Tool 목록
builtin_tools = {
    "calculator": "수식 계산",
    "get_current_time": "현재 시간 조회",
    "search_web": "웹 검색",
    "echo": "텍스트 반복",
}

print("🔧 내장 Tool:")
for tool_name, description in builtin_tools.items():
    print(f"  - {tool_name}: {description}")

# 사용 예제
print("\n💡 사용 예제:")
print("# calculator 사용")
print("result = calculator.run(expression='2 + 2')")
print("# get_current_time 사용")
print("time = get_current_time.run()")
print("# search_web 사용")
print("results = search_web.run(query='Python programming')")

## 2. 커스텀 Tool 생성

실제 사용 사례에 맞는 커스텀 Tool을 만들어봅시다.

### 2.1 간단한 Tool

기본적인 Tool 예제입니다.


In [ ]:
from beanllm import Tool


# 간단한 Tool 정의
def greet(name: str, language: str = "ko") -> str:
    """인사말을 생성합니다.

    Args:
        name: 이름
        language: 언어 ("ko", "en", "ja")

    Returns:
        인사말
    """
    greetings = {
        "ko": f"안녕하세요, {name}님!",
        "en": f"Hello, {name}!",
        "ja": f"こんにちは、{name}さん！",
    }
    return greetings.get(language, greetings["ko"])


# Tool 생성
greet_tool = Tool.from_function(greet)

print("✅ 커스텀 Tool 생성 완료!")
print(f"📊 이름: {greet_tool.name}")
print(f"📊 파라미터 스키마:")
for param_name, param_info in greet_tool.parameters.items():
    print(f"  - {param_name}: {param_info}")

# 실행
result = greet_tool.run(name="홍길동", language="ko")
print(f"\n💬 결과: {result}")

### 2.2 복잡한 Tool

여러 파라미터와 복잡한 로직을 가진 Tool입니다.


In [ ]:
from typing import List, Dict


@tool
def search_database(query: str, filters: Dict[str, str] = None, limit: int = 10) -> List[Dict]:
    """데이터베이스를 검색합니다.

    Args:
        query: 검색 쿼리
        filters: 필터 딕셔너리 (예: {"category": "tech", "year": "2024"})
        limit: 최대 결과 수

    Returns:
        검색 결과 리스트
    """
    # 실제로는 데이터베이스 쿼리
    results = [
        {"id": 1, "title": f"결과 1: {query}", "category": "tech"},
        {"id": 2, "title": f"결과 2: {query}", "category": "news"},
    ]
    return results[:limit]


print("✅ 복잡한 Tool 생성 완료!")
print(f"📊 파라미터 수: {len(search_database.parameters)}")

# 실행
results = search_database.run(query="Python", filters={"category": "tech"}, limit=5)
print(f"\n🔍 검색 결과: {len(results)}개")
for result in results:
    print(f"  - {result}")

## 3. Agent 기본 사용

Agent는 Tool을 사용하여 복잡한 작업을 수행할 수 있는 AI 에이전트입니다.

### 3.1 간단한 Agent

기본적인 Agent 사용법입니다.


In [ ]:
import asyncio
from beanllm import Agent, Tool


async def simple_agent():
    """간단한 Agent 예제"""

    # Tool 정의
    @tool
    def get_weather(city: str) -> str:
        """도시의 날씨를 조회합니다."""
        return f"{city}의 날씨는 맑습니다."

    # Agent 생성
    agent = Agent(
        model="gpt-4o",
        tools=[get_weather],
        max_iterations=5,
    )

    # Agent 실행
    result = await agent.run("서울의 날씨를 알려주세요.")

    print(f"✅ 최종 답변: {result.answer}")
    print(f"\n📊 실행 단계: {result.total_steps}단계")
    print(f"\n🔧 사용된 Tool:")
    for step in result.steps:
        if step.action:
            print(f"  - {step.action}: {step.action_input}")

    return result


# 실행 (주석 해제)
# result = await simple_agent()
print("💡 위의 주석을 해제하여 실행하세요.")

### 3.2 여러 Tool 사용

Agent는 여러 Tool을 조합하여 사용할 수 있습니다.


In [ ]:
async def multi_tool_agent():
    """여러 Tool을 사용하는 Agent"""

    # 여러 Tool 정의
    @tool
    def calculate(expression: str) -> str:
        """수식을 계산합니다."""
        try:
            return str(eval(expression))
        except:
            return "계산 오류"

    @tool
    def get_time() -> str:
        """현재 시간을 반환합니다."""
        from datetime import datetime

        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    @tool
    def format_text(text: str, style: str = "normal") -> str:
        """텍스트를 포맷팅합니다."""
        if style == "upper":
            return text.upper()
        elif style == "lower":
            return text.lower()
        return text

    # Agent 생성 (여러 Tool 포함)
    agent = Agent(
        model="gpt-4o",
        tools=[calculate, get_time, format_text],
        max_iterations=10,
    )

    # 복잡한 작업 실행
    result = await agent.run(
        "현재 시간을 가져와서 대문자로 포맷팅하고, " "그 다음에 2 + 2를 계산해주세요."
    )

    print(f"✅ 최종 답변: {result.answer}")
    print(f"\n📊 실행 단계:")
    for i, step in enumerate(result.steps, 1):
        print(f"\n{i}. Thought: {step.thought[:50]}...")
        if step.action:
            print(f"   Action: {step.action}")
            print(f"   Input: {step.action_input}")
        if step.observation:
            print(f"   Observation: {step.observation[:50]}...")


# 실행 (주석 해제)
# await multi_tool_agent()
print("💡 위의 주석을 해제하여 실행하세요.")

In [ ]:
async def react_example():
    """ReAct 패턴 예제"""

    @tool
    def search(query: str) -> str:
        """정보를 검색합니다."""
        # 실제로는 웹 검색 또는 데이터베이스 검색
        return f"{query}에 대한 검색 결과입니다."

    agent = Agent(
        model="gpt-4o",
        tools=[search],
        verbose=True,  # 상세 로그 출력
    )

    result = await agent.run("Python의 최신 버전은 무엇인가요?")

    print("\n📊 ReAct 단계별 분석:")
    for i, step in enumerate(result.steps, 1):
        print(f"\n{'='*60}")
        print(f"단계 {i}:")
        print(f"{'='*60}")
        print(f"💭 Thought: {step.thought}")
        if step.action:
            print(f"🔧 Action: {step.action}")
            print(f"📥 Input: {step.action_input}")
        if step.observation:
            print(f"👁️ Observation: {step.observation[:100]}...")
        if step.is_final:
            print(f"✅ Final Answer: {step.final_answer}")


# 실행 (주석 해제)
# await react_example()
print("💡 위의 주석을 해제하여 실행하세요.")

## 5. 고급 Tool 기능

### 5.1 Tool 검증

Tool의 입력을 검증할 수 있습니다.


In [ ]:
from beanllm import Tool, ToolParameter
from typing import Literal


# 검증이 포함된 Tool
def validate_age(age: int) -> str:
    """나이를 검증합니다.

    Args:
        age: 나이 (0-150 사이)

    Returns:
        검증 결과
    """
    if age < 0 or age > 150:
        return f"잘못된 나이: {age}"
    return f"유효한 나이: {age}세"


# Tool 생성 (파라미터 검증 포함)
age_tool = Tool.from_function(
    validate_age,
    parameters={
        "age": ToolParameter(
            type="integer",
            description="나이 (0-150)",
            minimum=0,
            maximum=150,
        )
    },
)

print("✅ 검증 기능이 있는 Tool 생성")
print(f"📊 파라미터 검증: age는 0-150 사이여야 함")

# 유효한 입력
result1 = age_tool.run(age=25)
print(f"\n✅ 유효한 입력: {result1}")

# 잘못된 입력 (검증 실패)
try:
    result2 = age_tool.run(age=200)
    print(f"❌ 잘못된 입력: {result2}")
except Exception as e:
    print(f"❌ 검증 실패: {e}")

### 5.2 Tool 체인

여러 Tool을 순차적으로 실행할 수 있습니다.


In [ ]:
from beanllm import ToolChain


# Tool 체인 예제
@tool
def fetch_data(url: str) -> str:
    """데이터를 가져옵니다."""
    return f"Data from {url}"


@tool
def process_data(data: str) -> str:
    """데이터를 처리합니다."""
    return f"Processed: {data.upper()}"


@tool
def save_data(data: str, filename: str) -> str:
    """데이터를 저장합니다."""
    return f"Saved {data} to {filename}"


# Tool 체인 생성
chain = ToolChain([fetch_data, process_data, save_data])

print("✅ Tool 체인 생성 완료!")
print("📊 체인 순서:")
for i, tool in enumerate(chain.tools, 1):
    print(f"  {i}. {tool.name}")

# 체인 실행
# result = chain.run(url="https://example.com", filename="output.txt")
# print(f"\n✅ 체인 실행 결과: {result}")

print("\n💡 Tool 체인은 여러 Tool을 순차적으로 실행합니다.")

## 6. 실전 예제

### 6.1 데이터 분석 Agent

여러 Tool을 사용하여 데이터를 분석하는 Agent입니다.


In [ ]:
async def data_analysis_agent():
    """데이터 분석 Agent"""

    @tool
    def load_csv(filepath: str) -> str:
        """CSV 파일을 로드합니다."""
        return f"CSV 데이터 로드됨: {filepath}"

    @tool
    def calculate_statistics(data: str) -> str:
        """통계를 계산합니다."""
        return "평균: 100, 최대: 200, 최소: 50"

    @tool
    def visualize(data: str, chart_type: str = "bar") -> str:
        """데이터를 시각화합니다."""
        return f"{chart_type} 차트 생성 완료"

    # Agent 생성
    agent = Agent(
        model="gpt-4o",
        tools=[load_csv, calculate_statistics, visualize],
        max_iterations=10,
    )

    # 작업 실행
    result = await agent.run(
        "data.csv 파일을 로드하고, 통계를 계산한 다음, " "막대 그래프로 시각화해주세요."
    )

    print(f"✅ 최종 답변: {result.answer}")
    print(f"\n📊 사용된 Tool:")
    for step in result.steps:
        if step.action:
            print(f"  - {step.action}")


# 실행 (주석 해제)
# await data_analysis_agent()
print("💡 위의 주석을 해제하여 실행하세요.")

### 6.2 웹 검색 Agent

웹 검색 Tool을 사용하는 Agent입니다.


In [ ]:
async def web_search_agent():
    """웹 검색 Agent"""

    from beanllm import search_web

    # Agent 생성 (웹 검색 Tool 포함)
    agent = Agent(
        model="gpt-4o",
        tools=[search_web, calculator],  # 웹 검색 + 계산기
        max_iterations=10,
    )

    # 복잡한 질문
    result = await agent.run(
        "최신 Python 버전을 검색하고, " "그 버전 번호에 2를 곱한 값을 계산해주세요."
    )

    print(f"✅ 최종 답변: {result.answer}")
    print(f"\n📊 실행 단계: {result.total_steps}단계")


# 실행 (주석 해제)
# await web_search_agent()
print("💡 위의 주석을 해제하여 실행하세요.")
print("⚠️ 웹 검색은 실제 API 호출이 필요합니다.")

## 📸 시각화 예제

### ReAct 패턴 시각화

> 💡 **이미지 필요**: ReAct 패턴의 흐름을 보여주는 다이어그램

```python
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(10, 6))

# ReAct 사이클
steps = ["Thought", "Action", "Observation", "Thought", "Action", "Final Answer"]

y_pos = 0
for i, step in enumerate(steps):
    x = i * 1.5
    color = "lightblue" if "Thought" in step else "lightgreen" if "Action" in step else "lightyellow"
    rect = mpatches.Rectangle((x, y_pos-0.3), 1.2, 0.6, facecolor=color, edgecolor='black')
    ax.add_patch(rect)
    ax.text(x+0.6, y_pos, step, ha='center', va='center', fontsize=10)
    
    if i < len(steps) - 1:
        ax.arrow(x+1.2, y_pos, 0.3, 0, head_width=0.1, head_length=0.1, fc='black', ec='black')

ax.set_xlim(-0.5, len(steps)*1.5)
ax.set_ylim(-1, 1)
ax.axis('off')
plt.title('ReAct 패턴 (Reasoning + Acting)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
```

**필요한 이미지**:
- 플로우차트: ReAct 패턴의 순환 구조
- Thought (파란색) → Action (초록색) → Observation (노란색) → Thought...
- 최종적으로 Final Answer 도달
- 화살표로 흐름 표시


## 7. 다음 단계

Agent와 Tools를 마스터했습니다! 다음 튜토리얼로 진행하세요:

### 📚 추천 학습 순서

1. **[06_chain_and_graph.ipynb](06_chain_and_graph.ipynb)** - Chain 및 Graph
   - 작업 체인 구성
   - 그래프 기반 워크플로우

2. **[07_multi_agent.ipynb](07_multi_agent.ipynb)** - Multi-Agent
   - 여러 Agent 협업
   - 에이전트 간 통신

3. **[11_web_search.ipynb](11_web_search.ipynb)** - Web Search
   - 웹 검색 통합
   - 실시간 정보 활용

### 🔗 관련 문서

- [API Reference](../API_REFERENCE.md#agent) - Agent API 상세 문서
- [README.md](../../README.md) - 프로젝트 개요

---

**✅ Agent & Tools 완료! 다음 노트북으로 진행하세요.**
